<a href="https://colab.research.google.com/github/ruitenbeek/thesis/blob/main/1step_svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
os.chdir('/content/gdrive/My Drive/thesis/code')
!pwd

/content/gdrive/My Drive/thesis/code


#Import

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from csv import DictReader
import pandas as pd
from sklearn.metrics import classification_report

#Read Files

In [ ]:
def read_file(file):
    data = list()
    abu_count = 0
    off_count = 0
    not_count = 0
    with open(file, 'r') as f:
        reader = DictReader(f, delimiter='\t')
        for row in reader:
            if (row['abusive'] == 'NOT' or row['abusive'] == 'UNKNOWN') and (row['explicitness'] == 'IMPLICIT' or row['explicitness'] == 'EXPLICIT'):
                data.append([row['text'], 'OFF'])
                off_count += 1
            elif row['abusive'] == 'IMPLICIT' or row['abusive'] == 'EXPLICIT':
                data.append([row['text'], 'ABU'])
                abu_count += 1
            elif (row['abusive'] == 'NOT' or row['abusive'] == 'UNKNOWN') and row['explicitness'] == 'NOT':
                data.append([row['text'], 'NOT'])
                not_count += 1
    print(f'ABU: %i\nOFF: %i\nNOT: %i' % (abu_count, off_count, not_count))
    data_df = pd.DataFrame(data)
    data_df.columns = ['text', 'label']
    return data_df


#Split Labels

In [ ]:
def split_labels(data_df):
    data_X = data_df.text.tolist()
    data_y = data_df.label.tolist()
    return data_X, data_y

#Model

In [ ]:
def svm_model(train_X, train_y):
    model = LinearSVC(random_state=0)
    model.fit(train_X, train_y)
    return model

#Evaluation

In [ ]:
def evaluation(model, test_X, test_y):
    pred_y = model.predict(test_X)
    target_names = ['ABU', 'NOT', 'OFF']
    print(classification_report(test_y, pred_y, target_names=target_names, digits=3))

# Main

In [ ]:
print('###TRAIN Split###')
train_data = read_file('train_final_pp.csv')
print('\n###DEV Split###')
dev_data = read_file('dev_final_pp.csv')
print('\n###TEST Split###')
test_data = read_file('test_final_pp.csv')
train_X, train_y = split_labels(train_data)
dev_X, dev_y = split_labels(dev_data)
test_X, test_y = split_labels(test_data)

all_tweets = train_X + dev_X + test_X 
vectorizer = TfidfVectorizer()
vectorizer.fit(all_tweets)
train_vectors = vectorizer.transform(train_X)
dev_vectors = vectorizer.transform(dev_X)
test_vectors = vectorizer.transform(test_X)

model = svm_model(train_vectors, train_y)
print('\n####DEV Results####')
evaluation(model, dev_vectors, dev_y)
print('\n####TEST Results####')
evaluation(model, test_vectors, test_y)

###TRAIN Split###
ABU: 1143
OFF: 1445
NOT: 5176

###DEV Split###
ABU: 110
OFF: 76
NOT: 361

###TEST Split###
ABU: 637
OFF: 399
NOT: 2072

####DEV Results####
              precision    recall  f1-score   support

         ABU      0.716     0.482     0.576       110
         NOT      0.814     0.934     0.870       361
         OFF      0.424     0.329     0.370        76

    accuracy                          0.759       547
   macro avg      0.651     0.581     0.605       547
weighted avg      0.740     0.759     0.741       547


####TEST Results####
              precision    recall  f1-score   support

         ABU      0.777     0.235     0.361       637
         NOT      0.785     0.962     0.864      2072
         OFF      0.309     0.291     0.300       399

    accuracy                          0.727      3108
   macro avg      0.624     0.496     0.508      3108
weighted avg      0.722     0.727     0.689      3108

